# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida, de llegada y duración
- Información de las escalas

¡Empecemos!
la pagina para mexico es <a href="https://www.latamairlines.com/mx/es">latamairlines mx</a>

debes entender que no puedes buscar vuelos a cualquier parte del mundo, ya que hay rutas establecidas, para encontrar una ruta turistica ve a la seccion de "ofertas de vuelo desde CDMX", asi encontre que rio es un pais muy visitado. 
el link que usaremos es para buscar vuelos con las siguientes caracteristicas:
- viaje solo de ida, economico, adulto, desde CDMX a rio de janeiro
- fecha 15 dias de la fecha actual osea para el 15 de febrero   
este es el link generado:   
https://www.latamairlines.com/mx/es/ofertas-vuelos?origin=MEX&inbound=null&outbound=2023-02-15T18%3A00%3A00.000Z&destination=RIO&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A


## ++ El poder de selenium 
A diferencia de usar requests solo para descargar el contenido de una pagina, selenium nos permite recibir y enviar datos mediante una interfaz en la cual interactua directamente con el navegador. Esto nos permite interctuar con las paginas y no solo leer su contenido. Esto es muy útil cuando queremos rellenar formularios, interactuar con contenido escondido mediante JS y mas. 
### Web element
El componente principal en selenium es el web element, que literalemente es un elemento selecionado de la pagina web generalmente una etiqueta. Sobre este podemos:
-	Hacer búsquedas: usando Xpath u lo mismos que beautiful soup
-	Insertar texto: para formularios
-	Hacer clik: para ingresar a otras pestañas.
Aquí tenemos algunos métodos:
https://www.browserstack.com/guide/selenium-webelement-commands


## inteto del objetivo usando requests y beautiful soup
intentaremos resolver la tarea usando lo aprendido en el anterior notebook 

In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.latamairlines.com/mx/es/ofertas-vuelos?origin=MEX&inbound=null&outbound=2023-02-15T18%3A00%3A00.000Z&destination=RIO&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A"
r = requests.get(url)
r.status_code

200

In [3]:
s = BeautifulSoup(r.text, 'lxml')
#print(s.prettify())

En la respuesta aparecera que habilitemos JS para ver todo el texto, esto es debido a que python no sabe interpretar JS. la respuesta no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

## Selenium
Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.
- Paso 1: instanciar un **driver** del navegador
    ```python
        from selenium import webdriver
        driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')
    ```
- Paso 2: hacer que el navegador cargue la página web.
    ```python
    driver.get(url)
    ```
- Paso 3: seleccionar una etiqeuta de la página. usando Xpath 
    ```python
    from selenium.webdriver.common.by import By
    vuelos = driver.find_elements(by=By.XPATH, value='//li[@class="sc-bvTASY cfqKKq"]')
    # traera todos las etiqeutas li con la class especicificada(serian todos los vuelos)
    ```
- paso 4: seleccionar un elemento de la etiqueta
    - texto(.text):
    ```python
    # texto de la duracion del vuelo
    duracion_vuelo = vuelo.find_element(by=By.XPATH, value ='.//div[@class="sc-klSiHT hjzFuR flight-duration"]/span[@class="sc-goFBvh kpzJsB"]').text 
    # si quisieramos un atriguto en vez de .text usariamos .get_attribute("name_atribute")
    ```
    - atributos(.get_attribute("name_atribute")):
    ```python
    # texto de la duracion del vuelo
    id_etiqueta_duracion_vuelo = vuelo.find_element(by=By.XPATH, value ='.//div[@class="sc-klSiHT hjzFuR flight-duration"]/span[@class="sc-goFBvh kpzJsB"]').get_attribute("id") 
    # si quisieramos un atriguto en vez de .text usariamos .get_attribute("name_atribute")
    ```
- Paso 5: cerrar el navegador
    ```python
    from selenium.webdriver.common.by import By
    vuelos = driver.find_elements(by=By.XPATH, value='//li[@class="sc-bvTASY cfqKKq"]')
    # traera todos las etiqeutas li con la class especicificada(serian todos los vuelos)
    ```

In [4]:
from selenium import webdriver

In [40]:
# definimos opciones para nuestro navedor que controlaremos desde selenium python
options = webdriver.ChromeOptions()
options.add_argument('--incognito') # indicamos que sea una ventana incognita "-private"

#usamos el driver para controlar una pestaña nueva de chrome desde el codigo de python mediante selenium
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')
driver.get(url) # en la pestaña nuva abrimos la pagina deseada 

C:\Users\Panda\AppData\Local\Temp\ipykernel_14504\3297477197.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')


### construyendo Xpath
a diferencia de beautiful soup, en selenium se usan Xpath como ruta para seleccionar una etiqeuta especifica.
el profesor menciona que para usar Xpath usemos la sintaxis 
```python
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
```
pero esta sintaxis ya no existe ahora se usa 

```python
from selenium.webdriver.common.by import By
vuelos = driver.find_elements(by=By.XPATH, value='//li[@class="sc-bvTASY cfqKKq"]')
#  o
vuelos = driver.find_element(by=By.XPATH, value='//li[@class="sc-bvTASY cfqKKq"]')

```

In [6]:
# Extraer la información de los vuelos
from selenium.webdriver.common.by import By
vuelos = driver.find_elements(by=By.XPATH, value='//li[@class="sc-bvTASY cfqKKq"]')
vuelo = vuelos[0]
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="1183e3f8c87ca8235bbddc4898ed4713", element="9d5b486f-d0c5-4148-863f-da120004d684")>

In [7]:
duracion_vuelo = vuelo.find_element(by=By.XPATH, 
value ='.//div[@class="sc-klSiHT hjzFuR flight-duration"]/span[@class="sc-goFBvh kpzJsB"]').text 
duracion_vuelo

'21 h 45 min'

In [8]:
# Dado que la información de la hora de salida y llegada se encuentra en diferentes 'div', pero tienen el mismo nombre de clase, por lo tanto, se debe utilizar el método '.find_elements()' para obtener mencionada información.

tiempo_salida_llegada = vuelo.find_elements(by=By.XPATH, 
value='.//div[@class="sc-klSiHT hjzFuR flight-information"]/span[@class="sc-hkHFWD lomdTu"]')

# Información de la hora de salida
tiempo_salida = tiempo_salida_llegada[0].text

# Información de la hora de llegada
tiempo_llegada = tiempo_salida_llegada[1].text

# Información de la duración del vuelo
duracion_vuelo = vuelo.find_element(by=By.XPATH, 
value ='.//div[@class="sc-klSiHT hjzFuR flight-duration"]/span[@class="sc-goFBvh kpzJsB"]').text 
# si quisieramos un atriguto en vez de .text usariamos .get_attribute("name_atribute")
print(tiempo_salida, tiempo_llegada)
print(duracion_vuelo)

4:00 p. m. 4:45 p. m.
+1
21 h 45 min


## interactuando con botones
selenium nos permite dar click a botones del navegador.  
en ocaciones el html del response html viene oculto, y se accede solo si damos click en una parte. si queremos acceder a ese contenido debemos saber interactuar con botones. 

In [9]:
boton_escalas = vuelo.find_element(by=By.XPATH, 
value ='.//a[@id="itinerary-modal-0-dialog-open"]') #buscamos el boton del enlace
boton_escalas.click() # damos clic sobre el enlace, el navegador cambiara

In [10]:
# ahora que ya tenemos una subpestaña abierta podemos interactuar con su html
# supongamos que queremos saber di hay escalas y donde sera
conecciones = vuelo.find_elements(by=By.XPATH, value ='//section[@data-test="section-info-connection"]')
print("numero de escalas: ", len(conecciones), "\n lugar de la escala: ")
conecciones[0].find_element(by=By.XPATH, value ='//span[@class="connection-text"]').text


numero de escalas:  1 
 lugar de la escala: 


'Conexión Lima'

ten en consideracion que cualquier boton del html lo podemos utilizar, asi podemos minimizar la subpestaña si conseguimos el xpath del boton de la X.   
te daras cuenta que cuando estas en el navegador y pasas encima del vuelo, te aparece la opcion de dar click. asi que la logica es la siguiente:
- busca una etiqueta en la cual puedas hacer click
- utiliza el metodo click

In [21]:
# seleccionamos el boton close de la subpestaña
vuelo.find_element(by=By.XPATH, value ='//button[@aria-label="Cerrar Itinerario de vuelo."]').click() # dara error si el navegador no tiene que cerrar

In [37]:
# damos click sobre el recuadro del vuelo para abrir las tarifas del vuelo
vuelo.find_element(by=By.XPATH, value ='//div[@role="button"]').click()
# ya adentro podemos acceder a las etiqeutas de las tarifas y transformarlas a float para posterior manipulacion
tarifas = vuelo.find_elements(by=By.XPATH, 
value ='//ol[@class="sc-buGlAa jhwXGF"]/li//span[@class="sc-eweMDZ ePnUhC displayAmount"]/span[@class="sc-ckYZGd grNCid"]')
[float(tarifa.text.replace(",","")) for tarifa in tarifas] 

[1480.55, 1663.55, 1869.55, 5328.55]

In [32]:
driver.close() # cerramos el navegador de enlace 

si alguién más obtuvo un status code 403: access denied lo resolví así:
```python
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(url, headers=agent)
```

## insertando contenido
algo peculiar de selenium es su capacidad de insertar datos. por ejemplo en esta ocacion nosotros insertaremos los datos del vuelo y fecha para buscar, todo desde selenium
use de guia: 
https://www.geeksforgeeks.org/how-to-simulate-pressing-enter-in-html-text-input-with-selenium/

In [41]:
root_url = "https://www.latamairlines.com/mx/es"
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')
driver.get(root_url)

C:\Users\Panda\AppData\Local\Temp\ipykernel_14504\101357599.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')


In [111]:
# seleccionamos toda la barra de busqueda
search_box = driver.find_element(by=By.XPATH, value ='//div[@id="MainSearchBoxDesktop"]')

# simulamos clicks para configurar vuelo solo de ida 
search_box.find_element(by=By.XPATH, value ='//button[@id="btnTripTypeCTA"]').click()
search_box.find_element(by=By.XPATH, value ='//button[@id="btnTripType0"]').click()

# ingresamos en texto de origen y destino y damos click
search_box.find_element(by=By.XPATH, value ='//input[@id="txtInputOrigin_field"]').send_keys("Ciudad de México")
search_box.find_element(by=By.XPATH, value ='//ul[@id="listAutoComplete"]/li[@id="lstItem_0"]').click() # requerimos dar click en una opcion valida
search_box.find_element(by=By.XPATH, value ='//input[@id="txtInputDestination_field"]').send_keys("Río de Janeiro")
search_box.find_element(by=By.XPATH, value ='//ul[@id="listAutoComplete"]/li[@id="lstItem_0"]').click()


# crear un formato de fecha valido para usar en el atributo aria-label de la fecha 
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_TIME, '') # importamos las fechas en español 

date_now = datetime.now()
mas_un_mes = date_now + timedelta(days=30) # le sumamos 30 dias a la fecha actual

dia_txt = mas_un_mes.strftime('%A') 
dia_num = mas_un_mes.day
mes_txt = mas_un_mes.strftime('%B')
año = mas_un_mes.year
fecha_formato = f"{dia_txt}, {dia_num} de {mes_txt} de {año}"

# ingresamos al campo de fecha y añadimos 
search_box.find_element(by=By.XPATH, value ='//input[@id="departureDate"]').click()
search_box.find_element(by=By.XPATH, value =f'//td[@aria-label="{fecha_formato}"]').click()

# click al link de busqueda
search_box.find_element(by=By.XPATH, value ='//button[@id="btnSearchCTA"]').click()

### uso de fechas
buscar en internet "como calcular que dia de la semana cae una fecha python"  

https://www.freecodecamp.org/espanol/news/modulo-datetime-de-python-como-manejar-fechas-en-python/#:~:text=C%C3%B3mo%20saber%20qu%C3%A9%20d%C3%ADa%20es&text=Podemos%20utilizar%20la%20funci%C3%B3n%20weekday,el%20martes%2C%20y%20as%C3%AD%20sucesivamente.

#### para traducir 
https://stackoverflow.com/questions/35226904/convert-spanish-date-in-string-format

####
para obtener los formatos de fechas 
https://help.gnome.org/users/gthumb/stable/gthumb-date-formats.html.en


## unificando el scraper 
en este caso crearemos algo con verdadera utiliadad. supongamos que quiero viajar dentro de un mes o mas a brasil, para lo cual necesito buscar entre 30 dias cual es el que mejor tarifas tiene. la pagina no muestra esta informacion directamente, asi que creare un scraper que analice para el siguiente mes varios dias y me diga cual es el dia que mejor precios tiene.
lo que hara el scrapper sera: 
1. 